In [1]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

import geopandas as gpd

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import requests

from os import listdir
import unicodedata

import json

from paths import *
from tools.vizu import remove_acentos
from tools.vizu import normalize_cols
from tools.io import read_sheets


from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')


import yaml

# Brasil.IO

In [2]:
####### IMPORT DATA ######
url = 'https://brasil.io/api/dataset/covid19/caso/data?format=json'
df_final = pd.DataFrame()

while url != None:
    
    print(url)
    response = requests.get(url)
    data = response.text
    parsed = json.loads(data)
    url = parsed['next']
    df = pd.DataFrame(parsed['results']).sort_values(by='confirmed',ascending=False)
    df_final = pd.concat([df_final,df], axis=0)

https://brasil.io/api/dataset/covid19/caso/data?format=json
https://brasil.io/api/dataset/covid19/caso/data?format=json&page=2
https://brasil.io/api/dataset/covid19/caso/data?format=json&page=3
https://brasil.io/api/dataset/covid19/caso/data?format=json&page=4


In [3]:
brio = df_final.copy()

mask = ((brio['is_last']==True) & (brio['place_type']=='city') & (brio['confirmed_per_100k_inhabitants'].notnull()))
cols = ['city_ibge_code','city','confirmed','deaths','date','state']
brio = brio[mask][cols]

## transformation

In [4]:
from tools.manipulation import create_br_not_sp

df = df_final.sort_values(by='confirmed',ascending=False)
dd = create_br_not_sp(df)

### GET ONLY LAST DAY
mask = dd['date'] < today
dd = dd[mask]

dd = dd.sort_values(by=['date'],ascending=False)
dd = dd.sort_values(by=['confirmed'],ascending=False)

# get only the states
mask = ((dd['place_type']!='city'))
df_states = dd[mask]
df_states = df_states.sort_values(by=['date'],ascending=False)

# Import Data

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv')
df['state'] = df['state'].str.replace('TOTAL','BRASIL')
df.to_csv('brasil_states.csv', index=False)
dd = df.copy()

In [6]:
dd = dd.drop(['country','deaths'],1)

In [7]:
# dd.tail(30)

## MYTABLE

In [8]:
from tools.io import read_sheets
df = read_sheets('covid19_estados')

In [9]:
# ddf = pd.merge(dd , df, on=['date','state'], how='left')
# ddf['confirmed'] = ddf['totalCases']
# ddf['new_cases'] = ddf['newCases']
# ddf = ddf.drop(['city','newCases','totalCases'],1)
# df  = ddf.copy()

In [10]:
for col in ['confirmed','new_cases','deaths','new_deaths']:
    df[col] = pd.to_numeric(df[col])

In [11]:
# mask = df['new_deaths'].notnull()
# df = df[mask]

In [12]:
df_states = df.sort_values(by=['date','confirmed'], ascending=False)
df_states['city'] = df_states['state']
df_states.head()

mask = (df_states['state']!='BRASIL') & (df_states['state']!='SP')
not_sp = df_states[mask].groupby(by=['date'], as_index=False).sum()

not_sp['state'] = 'BRASIL SEM SP'
not_sp['city'] = 'BRASIL SEM SP'
df_states = pd.concat([df_states,not_sp[df_states.columns]],axis=0)


## VIS

In [13]:
from tools.vizu import brasil_vis
from tools import io

themes = yaml.load(open('../themes/custom_colorscales.yaml', 'r'), Loader=yaml.FullLoader)

df_states = df_states.sort_values(by=['date'],ascending=False)

escala  = 'lin'
save    = True
#### CHOSE CITIES TO START SELECTED IN GRAPH
in_cities = ['BRASIL','BRASIL SEM SP','SP', 'RJ']

# mask = ((dd[var_col]>0))
# dd = dd[mask]

## confirmed or deaths
for var in ['confirmed','deaths']:
    fig = brasil_vis(df_states,
                     var,
                     in_cities,
                     today='31/03/2020',
                     save=True,
                     themes=themes['novo'])
    
    
    name= "brasil_por_estado_{}.html".format(var)
    path= f"../images/brasil/{name}"
    
#     plot(fig, filename=path, auto_open=False)
# #     plot(fig, filename="../../sample_pages/images/covid19/brasil/brasil_por_estado_{}.html".format(var_col), auto_open=False)
    
    io.to_storage(bucket='sv-covid19',
                  bucket_folder='brasil',
                  file_name=name,
                  path_to_file=path)



Done!
Done!


In [14]:
# fig

### Figures to UP

In [15]:
f1_name = 'total_lin.html'
f2_name = 'mortes_lin.html'
f3_name = 'brasil_lin_total.html'
f4_name = 'brasil_lin_mortes.html'
# f5_name = "brasil_por_estado_confirmed.html"
# f6_name = "brasil_por_estado_deaths.html"

f1_path = f"../images/multipleCountry/{f1_name}"
f2_path = f"../images/multipleCountry/{f2_name}"
f3_path = f"../images/singleCountry/{f3_name}"
f4_path = f"../images/singleCountry/{f4_name}"
# f5_path = f"../images/brasil/{f5_name}"
# f6_path = f"../images/brasil/{f6_name}"

names_paths = {f1_name:f1_path,
         f2_name:f2_path,
         f3_name:f3_path,
         f4_name:f4_path
              }
#          f5_name:f5_path,
#          f6_name:f6_path}

for name in names_paths.keys():
    
    io.to_storage(bucket='sv-covid19',
              bucket_folder='brasil',
              file_name=name,
              path_to_file=names_paths[name])

Done!
Done!
Done!
Done!


## MAPS

In [16]:
maps = gpd.read_file('../../mapas_brasil/municipios-ibge/brasil-municipios.shp')
ufs = gpd.read_file('../../mapas_brasil/estados-ibge/BRUFE250GC_SIR.shp')

In [17]:
# df = pd.read_csv('https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities.csv', dtype={'ibgeID':str})
df = brio.copy()

municipios = pd.read_csv('../data/br_municipios_ibge.csv', dtype={'geocodigo':str})

cols = ['geocodigo','state','city','nome_uf','nome_mesorregiao','nome_municipio','confirmed','deaths','date']
dd = pd.merge(df,municipios, left_on='city_ibge_code', right_on='geocodigo', how = 'left')[cols]

final = pd.merge(dd,maps, left_on='geocodigo', right_on='CD_GEOCMU', how='left').drop(['CD_GEOCMU','NM_MUNICIP'],1)

In [18]:
final =  final[final['geometry'].notnull()]
final = gpd.GeoDataFrame(final)
final.crs = {'init' :'epsg:4326'}

rename_cols = {'nome_municipio':'Município','nome_uf':'Estado','confirmed':'Casos','deaths':'Óbitos', 'date':'Data do Boletim'}
final = final.rename(columns=rename_cols)

In [19]:
mask = df_states['date'] == max(df_states['date'])
estados = df_states[mask]


ff = final[['state','Estado']].drop_duplicates()
estados = pd.merge(estados,ff,on='state', how='inner')

estados['NM_ESTADO'] = estados['Estado'].str.upper()
estados = pd.merge(estados,ufs, on = 'NM_ESTADO')

estados = gpd.GeoDataFrame(estados)
estados.crs = {'init' :'epsg:4326'}

In [20]:
rename_cols = {'confirmed':'Casos','deaths':'Óbitos','new_cases':'Novos Casos','new_deaths':'Novas Mortes','date':'Data do Boletim'}
estados = estados.rename(columns=rename_cols)

In [21]:
from scripts import vis_maps
mymap =  vis_maps.get_map(final,'Casos', estados)

mymap.save('../maps/cidades_estados.html')
# mymap.save('../../sample_pages/images/covid19/maps/cidades_estados.html')

In [22]:
from scripts import io

io.to_storage(bucket='sv-covid19',
              bucket_folder='maps',
              file_name='cidades_estados.html',
              path_to_file='../maps/cidades_estados.html')

Done!


## Vale

In [23]:
# estado_sp = estados[estados['state']=='SP']
estado_sp = estados.copy()
col_rename ={
    'Casos':'Casos Confirmados',
    'Óbitos':'Óbitos Confirmados',
    'Data':'Data do Boletim',
}

estado_sp = estado_sp.rename(columns=col_rename)

sp = gpd.read_file('../../brasil_geodata/maps/sp_municipios.json')

In [24]:
from tools.io import read_sheets
df = read_sheets('covid19_vale_do_paraiba')

df['nome_municipio'] = df['municipio']

cols = ['suspeitas','suspeitas_internados','confirmados','descartados','mortes','recuperados']
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
df = df.fillna('-').replace('','-')

In [25]:
vale = pd.merge(df,sp[['nome_municipio','geometry']],on='nome_municipio', how='left')
vale = vale[vale['geometry'].notnull()]
vale = gpd.GeoDataFrame(vale)
vale.crs = {'init' :'epsg:4326'}

col_rename ={
    'municipio':'Município',
    'confirmados':'Casos Confirmados',
    'mortes':'Óbitos Confirmados',
    'ultimo_boletim':'Data do Boletim',
    'fonte':'Fonte'   
}

vale = vale.rename(columns=col_rename)

In [26]:
from scripts import vis_maps

cols = ['Município','Casos Confirmados', 'Óbitos Confirmados','Data do Boletim']
mymap =  vis_maps.get_map_vale(vale,'Casos Confirmados', cols, estado_sp)
mymap.save('../maps/mapa_vale.html')


In [27]:
from scripts import io

io.to_storage(bucket='sv-covid19',
              bucket_folder='maps',
              file_name='mapa_vale.html',
              path_to_file='../maps/mapa_vale.html')

Done!


In [77]:


# title  = 'Óbitos por Faixa Etária'
# x_name = 'Óbitos'
# y_name = 'Faixa Etária'

# trace = go.Bar(
#     orientation='h',
#     x = df['obitos'],
#     y = df['faixa']
# )

# data = [trace]

# layout = go.Layout(
#     barmode='stack',
    
#     yaxis_title=y_name,
#     yaxis = dict(
#         tickfont=dict(
#             size=22,
#             color='black',
#         ),
#     ),
#     xaxis_title=x_name,
#     xaxis = dict(
#         tickfont=dict(
#             size=22,
#             color='black',
#         ),
# #         font = dict(size=20)

#     ),
    
#     font=dict(
#             size=25,
#         ),

#     width  = 800,
#     height = 800,
# )

# fig = go.Figure(data=data, layout=layout)
# fig